In [1]:
season = 2024
max_week = 9
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
from torch import nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

In [2]:
filename = os.path.join(os.path.join("Data", str(season)),"plays.csv")
plays = pd.read_csv(filename)
filename = os.path.join(os.path.join("Data", str(season)),"players.csv")
players = pd.read_csv(filename)
#print(sum(plays['passLength'] > 0))
x = []
y = []

In [14]:
import warnings
n = 46
warnings.simplefilter('ignore',lineno=n + 1)
warnings.simplefilter('ignore',lineno=n+2)
warnings.simplefilter('ignore',lineno=n+3)
warnings.simplefilter('ignore',lineno=n+4)
warnings.simplefilter('ignore',lineno=n+5)
warnings.simplefilter('ignore',lineno=n+6)
num = 0
#x = []
#y = []
#state = []
#playerslsx = []
#playerlsy = []
pause = 48000
x = x[:pause]
y = y[:pause]
state = state[:pause]
playerslsx = playerslsx[:pause]
playerlsy = playerlsy[:pause]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
for i in range(1,max_week + 1):
    filename = os.path.join(os.path.join("Data", str(season)),"tracking_week_"+str(i) +".csv")
    #print(filename)
    week = pd.read_csv(filename)
    #print(np.unique(week['event'].astype(str)))
    #print(np.unique(week["event"]))
    #print(np.unique(week['event'].asty))
    #print(len((week['playId'])))
    '''sum1 = 0
    for g in np.unique(week['gameId']):
            #print(g)
            game = week[week['gameId'] == g]
            for p in np.unique(game['playId']):
                play = game[game["playId"] == p]
                if 'pass_outcome_caught' in play['event'].astype(str).tolist():
                    sum1 += 1
    print(sum, sum())'''
    for g in np.unique(week['gameId']):
        #print(g)
        game = week[week['gameId'] == g]
        for p in np.unique(game['playId']):
            play = game[game["playId"] == p]
            #print(play['event'])
            #ev = np.unique(game[game["playId"] == p]['event'].astype(str))
            if play.iloc[0,8] == "left":
                play['x'] = (120 - play['x'].values)
                play['y'] = (53 - play['y'].values)
                play['dir'] = (play['dir'].values + 180)
                play['o'] = (play['o'].values + 180)
                play['dir'] = (play['dir'].values - (play['dir'] > 360)*360)
                play['o'] = (play['o'].values - (play['o'] > 360)*360)
            
            carrierId = plays[(plays['gameId'] == g) & (plays['playId'] == p)]["ballCarrierId"].iloc[0]
            carrierx = play[play["nflId"] == carrierId]['x']
            
            cont = True
            minframe = 1
            if len(carrierx) < 1:
                cont = False
            else:
                #maxyds = max(carrierx)
                maxyds = carrierx.iloc[-1]
            #desc = play['event'].astype(str).tolist()
            cont = len(play[(play['event'] == 'fumble')]) == 0
            minframe = play[(play['event'] == 'pass_outcome_caught') | (play['event'] == 'handoff') |(play['event'] == 'lateral') |(play['event'] == 'run') |(play['event'] == 'run_pass_option')]['frameId']
            if len(minframe) > 0:
                minframe = minframe.iloc[-1]
            else:
                minframe = 1
            '''if 'fumble' in desc:
                cont = False
            elif 'pass_outcome_caught' in desc:
                minframe = play[play['event'] == 'pass_outcome_caught']['frameId'].iloc[0]
                #print("yo")
            elif 'handoff' in desc:
                minframe = play[play['event'] == 'handoff']['frameId'].iloc[0]
            elif 'lateral' in desc:
                minframe = play[play['event'] == 'lateral']['frameId'].iloc[0]
            elif 'run' in desc:
                minframe = play[play['event'] == 'run']['frameId'].iloc[0]
            elif 'run_pass_option' in desc:
                minframe = play[play['event'] == 'run_pass_option']['frameId'].iloc[0]'''
            
            if plays[(plays['gameId'] == g) & (plays["playId"] == p)]['playNullifiedByPenalty'].iloc[0] == 'Y':
                cont = False
            team1 = plays[(plays['gameId'] == g) & (plays['playId'] == p) ]['possessionTeam'].iloc[0]
            if minframe == 1 and cont:
                for f in np.unique(play['frameId']): 
                    if num >= pause:

                        state += [[g,p,f]]
                        
                        #print(minframe)
                        frame = play[play['frameId'] == f]
                        
                        ball = frame[frame["displayName"] == 'football']
                        ballind = ball.index
                        frame = frame.drop(ballind)
                        carrierId = plays[(plays['gameId'] == g) & (plays['playId'] == p)]["ballCarrierId"].iloc[0]
                        carrier = frame[frame["nflId"] == carrierId]

                        frame = pd.concat([frame.drop(carrier.index), carrier])
                        numplayers = len(np.unique(play['nflId']))   
                        temp = torch.zeros(8,numplayers + 1, numplayers - 1)
                        #temp[temp == 0] = np.nan
                        j = 0
                        playerlsy += [frame['nflId']]
                        #print(frame[frame["nflId"] == carrierId]['x'], carrier)
                        ballx = frame[frame["nflId"] == carrierId]['x']
                        #bally = frame[frame["displayName"] == 'football']['y'].iloc[0]
                        if len(ballx) > 0:
                            ballx = ballx.iloc[0]
                            for player1 in np.unique(play['nflId']):
                                k = 0
                                #ballx = ballx.iloc[0]

                                if not np.isnan(player1):
                                    x1 = frame[frame['nflId'] == player1]['x'].iloc[0]
                                    y1 = frame[frame['nflId'] == player1]['y'].iloc[0]
                                    s1 = frame[frame['nflId'] == player1]['s'].iloc[0]
                                    a1 = frame[frame['nflId'] == player1]['a'].iloc[0]
                                    o1 = frame[frame['nflId'] == player1]['o'].iloc[0]
                                    dir1 = frame[frame['nflId'] == player1]['dir'].iloc[0]
                                    team2 = frame[frame['nflId'] == player1]['club'].iloc[0]
                                    teamvec1 = frame['club'] == team1
                                    teamvec2 = frame['club'] == team2
                                    temp[7,0:-2,j] = torch.tensor((teamvec2*(((frame['x'] - x1)**2 + (frame['y'] - y1)**2)**0.5) - (~teamvec2)*(((frame['x'] - x1)**2 + (frame['y'] - y1)**2)**0.5)).values)
                            
                                    temp[0,0:-2,j] = torch.tensor((frame['x'] - x1).values)
                                    temp[1,0:-2,j] = torch.tensor((frame['y'] - y1).values)
                                    temp[2,0:-2,j] = torch.tensor((frame['s'] - s1).values)
                                    temp[3,0:-2,j] = torch.tensor((frame['a'] - a1).values)
                                    temp[4,0:-2,j] = torch.tensor((frame['dir'] - dir1).values)
                                    temp[4,0:-2,j] = temp[3,0:-2,j] - (temp[3,0:-2,j] > 360)*-360
                                    temp[4,0:-2,j] = temp[3,0:-2,j] + (temp[3,0:-2,j] < 0)*-360
                                    #print((np.arctan2((frame['y'] - y1).to_numpy(), (frame['x'] - x1).to_numpy()) - o1))
                                    temp[5,0:-2,j] = torch.tensor((np.arctan2((frame['y'] - y1).to_numpy(), -1*(frame['x'] - x1).to_numpy())*180/np.pi))
                                    #print((((-1*(frame['x'] - x1) > 0).values) & ((frame['y'] - y1) < 0)).values)
                                    temp[5,0:-2,j] = temp[4,0:-2,j] + (1*(-1*(frame['x'] - x1) > 0).values & (((frame['y'] - y1) < 0)*180).values) + (-1*(-1*(frame['x'] - x1) > 0).values & ((frame['y'] - y1) < 0).values)*180
                                    temp[5,0:-2,j] = temp[4,0:-2,j] -360*(temp[4,0:-2,j] > 360)
                                    temp[5,0:-2,j] = temp[4,0:-2,j] +360*(temp[4,0:-2,j] < 0)
                                    temp[5,0:-2,j] -= o1
                                    temp[6,0:-2,j] = torch.tensor(teamvec1.values)
                                    '''for player2 in np.unique(play['nflId']):
                                        if not np.isnan(player2):
                                            x2 = frame[frame['nflId'] == player2]['x'].iloc[0]
                                            y2 = frame[frame['nflId'] == player2]['y'].iloc[0]
                                            s2 = frame[frame['nflId'] == player2]['s'].iloc[0]
                                            a2 = frame[frame['nflId'] == player2]['a'].iloc[0]
                                            o2 = frame[frame['nflId'] == player2]['o'].iloc[0]
                                            dir2 = frame[frame['nflId'] == player2]['dir'].iloc[0]
                                            team2 = frame[frame['nflId'] == player2]['club'].iloc[0]
                                            if team1 == team2:
                                                temp[0,k,j] = ((x1-x2)**2 + (y1-y2)**2)**0.5
                                            else:
                                                temp[0,k,j] = -1*((x1-x2)**2 + (y1-y2)**2)**0.5
                                            temp[1,k,j] = s2-s1
                                            temp[2,k,j] = a2-a1
                                            temp[3,k,j] = dir2-dir1
                                            #print(np.arctan(y2-y1,x2-x1))
                                            temp[4,k,j] = np.arctan2(y2-y1,x2-x1) - o1
                                            k += 1
        '''
                                    if y1 >= 53.3/2:
                                        temp[0,-2,j] = 53.3 - y1
                                    else:
                                        temp[0,-2,j] = y1
                                    
                                    temp[0,-1,j] = 105 - x1
                                    #temp[0,-3,j] = ((x1-ballx)**2 + (y1-bally)**2)**0.5
                                    j += 1
                                    #print(j,temp)

                            #print(temp[0,-3,:].sort()[1])
                            inds = temp[7,-3,:].sort()[1]
                            temp = temp[:,:,inds]
                            #print(playerlsy[-1])
                            #playerslsx += [(playerlsy[-1]).iloc[temp[7,-3,:].sort()[1]]]
                            playerslsx += [playerlsy[-1].iloc[ind] for ind in inds.tolist()]
                            #print(temp[0,-3,:])
                            x += [temp]
                            if num % 1000 == 0:
                                print(num)
                            #print(num)
                            num += 1
                            
                            y += [maxyds - ballx]
                    else:
                        num += 1
                        if num % 1000 == 0:
                            print(num)
                        

            elif cont:
                for f in np.unique(play[(play["frameId"] >= minframe)]['frameId']):
                    #print(minframe)
                    if num >= pause:

                        state += [[g,p,f]]
                        
                        #print(minframe)
                        frame = play[play['frameId'] == f]
                        
                        ball = frame[frame["displayName"] == 'football']
                        ballind = ball.index
                        frame = frame.drop(ballind)
                        carrierId = plays[(plays['gameId'] == g) & (plays['playId'] == p)]["ballCarrierId"].iloc[0]
                        carrier = frame[frame["nflId"] == carrierId]

                        frame = pd.concat([frame.drop(carrier.index), carrier])
                        numplayers = len(np.unique(play['nflId']))   
                        temp = torch.zeros(8,numplayers + 1, numplayers - 1)
                        #temp[temp == 0] = np.nan
                        j = 0
                        playerlsy += [frame['nflId']]
                        #print(frame[frame["nflId"] == carrierId]['x'], carrier)
                        ballx = frame[frame["nflId"] == carrierId]['x']
                        #bally = frame[frame["displayName"] == 'football']['y'].iloc[0]
                        if len(ballx) > 0:
                            ballx = ballx.iloc[0]
                            for player1 in np.unique(play['nflId']):
                                k = 0
                                

                                if not np.isnan(player1):
                                    x1 = frame[frame['nflId'] == player1]['x'].iloc[0]
                                    y1 = frame[frame['nflId'] == player1]['y'].iloc[0]
                                    s1 = frame[frame['nflId'] == player1]['s'].iloc[0]
                                    a1 = frame[frame['nflId'] == player1]['a'].iloc[0]
                                    o1 = frame[frame['nflId'] == player1]['o'].iloc[0]
                                    dir1 = frame[frame['nflId'] == player1]['dir'].iloc[0]
                                    team2 = frame[frame['nflId'] == player1]['club'].iloc[0]
                                    teamvec1 = frame['club'] == team1
                                    teamvec2 = frame['club'] == team2
                                    temp[7,0:-2,j] = torch.tensor((teamvec2*(((frame['x'] - x1)**2 + (frame['y'] - y1)**2)**0.5) - (~teamvec2)*(((frame['x'] - x1)**2 + (frame['y'] - y1)**2)**0.5)).values)
                            
                                    temp[0,0:-2,j] = torch.tensor((frame['x'] - x1).values)
                                    temp[1,0:-2,j] = torch.tensor((frame['y'] - y1).values)
                                    temp[2,0:-2,j] = torch.tensor((frame['s'] - s1).values)
                                    temp[3,0:-2,j] = torch.tensor((frame['a'] - a1).values)
                                    temp[4,0:-2,j] = torch.tensor((frame['dir'] - dir1).values)
                                    temp[4,0:-2,j] = temp[3,0:-2,j] - (temp[3,0:-2,j] > 360)*-360
                                    temp[4,0:-2,j] = temp[3,0:-2,j] + (temp[3,0:-2,j] < 0)*-360
                                    #print((np.arctan2((frame['y'] - y1).to_numpy(), (frame['x'] - x1).to_numpy()) - o1))
                                    temp[5,0:-2,j] = torch.tensor((np.arctan2((frame['y'] - y1).to_numpy(), -1*(frame['x'] - x1).to_numpy())*180/np.pi))
                                    #print((((-1*(frame['x'] - x1) > 0).values) & ((frame['y'] - y1) < 0)).values)
                                    temp[5,0:-2,j] = temp[4,0:-2,j] + (1*(-1*(frame['x'] - x1) > 0).values & (((frame['y'] - y1) < 0)*180).values) + (-1*(-1*(frame['x'] - x1) > 0).values & ((frame['y'] - y1) < 0).values)*180
                                    temp[5,0:-2,j] = temp[4,0:-2,j] -360*(temp[4,0:-2,j] > 360)
                                    temp[5,0:-2,j] = temp[4,0:-2,j] +360*(temp[4,0:-2,j] < 0)
                                    temp[5,0:-2,j] -= o1
                                    temp[6,0:-2,j] = torch.tensor(teamvec1.values)
                                    '''for player2 in np.unique(play['nflId']):
                                        if not np.isnan(player2):
                                            x2 = frame[frame['nflId'] == player2]['x'].iloc[0]
                                            y2 = frame[frame['nflId'] == player2]['y'].iloc[0]
                                            s2 = frame[frame['nflId'] == player2]['s'].iloc[0]
                                            a2 = frame[frame['nflId'] == player2]['a'].iloc[0]
                                            o2 = frame[frame['nflId'] == player2]['o'].iloc[0]
                                            dir2 = frame[frame['nflId'] == player2]['dir'].iloc[0]
                                            team2 = frame[frame['nflId'] == player2]['club'].iloc[0]
                                            if team1 == team2:
                                                temp[0,k,j] = ((x1-x2)**2 + (y1-y2)**2)**0.5
                                            else:
                                                temp[0,k,j] = -1*((x1-x2)**2 + (y1-y2)**2)**0.5
                                            temp[1,k,j] = s2-s1
                                            temp[2,k,j] = a2-a1
                                            temp[3,k,j] = dir2-dir1
                                            #print(np.arctan(y2-y1,x2-x1))
                                            temp[4,k,j] = np.arctan2(y2-y1,x2-x1) - o1
                                            k += 1
        '''
                                    if y1 >= 53.3/2:
                                        temp[0,-2,j] = 53.3 - y1
                                    else:
                                        temp[0,-2,j] = y1
                                    
                                    temp[0,-1,j] = 105 - x1
                                    #temp[0,-3,j] = ((x1-ballx)**2 + (y1-bally)**2)**0.5
                                    j += 1
                                    #print(j,temp)

                            #print(temp[0,-3,:].sort()[1])
                            inds = temp[7,-3,:].sort()[1]
                            temp = temp[:,:,inds]
                            #print(playerlsy[-1])
                            #playerslsx += [(playerlsy[-1]).iloc[temp[7,-3,:].sort()[1]]]
                            playerslsx += [playerlsy[-1].iloc[ind] for ind in inds.tolist()]
                            #print(temp[0,-3,:])
                            x += [temp]
                            if num % 1000 == 0:
                                print(num)
                            #print(num)
                            num += 1
                            
                            y += [maxyds - ballx]
                    else:
                        num += 1
                        if num % 1000 == 0:
                            print(num)
                    

torch.save(x,'locdatagood24-2.pt')
torch.save(y,'ydsweekgood24-2.pt')
torch.save(state,'stateofgood-2.pt')
torch.save(playerlsy,'playersgoody-2.pt')
torch.save(playerslsx,'playersgoodx-2.pt')


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
48000


In [ ]:
torch.save(x,'locdatagood24-2.pt')
torch.save(y,'ydsweekgood24-2.pt')
torch.save(state,'stateofgood-2.pt')
torch.save(playerlsy,'playersgoody-2.pt')
torch.save(playerslsx,'playersgoodx-2.pt')

In [ ]:
num = 0
x = []
y = []
state = []
playerslsx = []
playerlsy = []
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
for i in range(1,max_week + 1):
    filename = os.path.join(os.path.join("Data", str(season)),"tracking_week_"+str(i) +".csv")
    #print(filename)
    week = pd.read_csv(filename)
    print(np.unique(week['event'].astype(str)))
    #print(np.unique(week["event"]))
    #print(np.unique(week['event'].asty))
    #print(len((week['playId'])))
    '''sum1 = 0
    for g in np.unique(week['gameId']):
            #print(g)
            game = week[week['gameId'] == g]
            for p in np.unique(game['playId']):
                play = game[game["playId"] == p]
                if 'pass_outcome_caught' in play['event'].astype(str).tolist():
                    sum1 += 1
    print(sum, sum())'''
    for g in np.unique(week['gameId']):
        #print(g)
        game = week[week['gameId'] == g]
        for p in np.unique(game['playId']):
            play = game[game["playId"] == p]
            #print(play['event'])
            #ev = np.unique(game[game["playId"] == p]['event'].astype(str))
            if play.iloc[0,8] == "left":
                play['x'] = (120 - play['x']).values
                play['y'] = (53 - play['y']).values
                play['dir'] = (play['dir'] + 180).values
                play['o'] = (play['o'] + 180).values
                play['dir'] = (play['dir'] - (play['dir'] > 360)*360).values
                play['o'] = (play['o'] - (play['o'] > 360)*360).values
            
            carrierId = plays[(plays['gameId'] == g) & (plays['playId'] == p)]["ballCarrierId"].iloc[0]
            carrierx = play[play["nflId"] == carrierId]['x']
            
            cont = True
            minframe = 1
            if len(carrierx) < 1:
                cont = False
            else:
                #maxyds = max(carrierx)
                maxyds = carrierx.iloc[-1]
            if 'pass_outcome_caught' in play['event'].astype(str).tolist():
                minframe = play[play['event'] == 'pass_outcome_caught']['frameId'].iloc[0]
                #print("yo")
            
            if plays[(plays['gameId'] == g) & (plays["playId"] == p)]['playNullifiedByPenalty'].iloc[0] == 'Y':
                minframe = 100000
            team1 = plays[(plays['gameId'] == g) & (plays['playId'] == p) ]['possessionTeam'].iloc[0]
            if minframe == 1 and cont:
                for f in np.unique(play['frameId']): 
                    
                    #print(minframe)
                    frame = play[play['frameId'] == f]
                    
                    
                    #temp[temp == 0] = np.nan
                    num +=1 
                    
                    ballx = frame[frame["nflId"] == carrierId]['x'].iloc[0]
                    #bally = frame[frame["displayName"] == 'football']['y'].iloc[0]
                    print(num)
                    y += [maxyds - ballx]
                        

            elif cont:
                for f in np.unique(play[(play["frameId"] >= minframe)]['frameId']):
                    #print(minframe)
                    frame = play[play['frameId'] == f]
                    
                    
                    #temp[temp == 0] = np.nan
                    num +=1 
                    
                    ballx = frame[frame["nflId"] == carrierId]['x'].iloc[0]
                    #bally = frame[frame["displayName"] == 'football']['y'].iloc[0]
                    print(num)
                    y += [maxyds - ballx]

torch.save(y,'ydsadjgood24.pt')
                    

In [7]:
torch.save(y,'ydsadjgood24.pt')


In [23]:
torch.save(x,'locdatagood24.pt')
torch.save(y,'ydsweekgood24.pt')
torch.save(state,'stateofgood.pt')
torch.save(playerlsy,'playersgoody.pt')
torch.save(playerslsx,'playersgoodx.pt')



In [17]:
print(len(week['gameId']))
[xtrain,xtest, ytrain, ytest] = train_test_split(x,y, shuffle=True)
xtrain[1].shape

1407439


torch.Size([5, 25, 22])

In [23]:
# Creating a CNN class
class ConvNeuralNet(nn.Module):
	#  Determine what layers and their order in CNN object 
    def __init__(self, num_classes):
        super(ConvNeuralNet, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)
        self.conv_layer2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.conv_layer3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv_layer4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
        self.max_pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        
        self.fc1 = nn.Linear(1600, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)
    
    # Progresses data across layers    
    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.conv_layer2(out)
        out = self.max_pool1(out)
        
        out = self.conv_layer3(out)
        out = self.conv_layer4(out)
        out = self.max_pool2(out)
                
        out = out.reshape(out.size(0), -1)
        
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out
    
model = ConvNeuralNet()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
 
n_epochs = 20
for epoch in range(n_epochs):
    for i in range(len(ytrain)):
        # forward, backward, and then weight update
        y_pred = model(xtrain[i])
        loss = loss_fn(y_pred, ytrain[i])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
 
    acc = 0
    count = 0
    for i in range(len(ytest)):
        y_pred = model(xtest[i])
        acc += ((y_pred - ytest[i]) <= 0.1)
        count += 1
    acc /= count
    print("Epoch %d: model accuracy %.2f%%" % (epoch, acc*100))
 
torch.save(model.state_dict(), "cifar10model.pth")

RuntimeError: Given groups=1, weight of size [22, 25, 3, 3], expected input[1, 22, 25, 22] to have 25 channels, but got 22 channels instead